# ⚡ MIR GPU BATCH-TURBO: Procesamiento Masivo Paralelo
### *"The Ultimate Matrix Shredder"*

Este cuaderno implementa la ventaja definitiva de las GPUs: el **Procesamiento por Lotes (Batch Processing)**. 

**¿Cómo funciona?**
1. **Agrupación**: Cargamos múltiples archivos simultáneamente.
2. **Padding**: Rellenamos las pistas con silencio para que todas tengan la misma longitud espectral.
3. **Saturación CUDA**: Enviamos un bloque masivo de datos a los núcleos de la GPU, calculando las transformadas de varias canciones en un solo ciclo de reloj.

---

In [ ]:
!pip install -q nnAudio torchaudio torch pandas matplotlib tqdm

import os
import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shutil
import time
import warnings
from nnAudio import features
from IPython.display import Audio, display, FileLink
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
import librosa
import librosa.display

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 MODO BATCH ACTIVADO: {device.upper()}")

plt.style.use('dark_background')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (16, 8)})

OUTPUT_DIR = 'batch_gpu_results'
os.makedirs(os.path.join(OUTPUT_DIR, 'plots'), exist_ok=True)

## ⚡ 1. Inicialización de Kernels Paralelos

In [ ]:
SAMPLE_RATE = 44100
BATCH_SIZE = 8 # Número de canciones procesadas simultáneamente

mel_gpu = features.MelSpectrogram(sr=SAMPLE_RATE, n_fft=2048, n_mels=128).to(device)
cqt_gpu = features.CQT2010v2(sr=SAMPLE_RATE, hop_length=512, fmin=32.7, n_bins=84).to(device)

print(f"✨ Motores listos para procesar lotes de {BATCH_SIZE} pistas.")

## 🧬 2. El Motor de Lotes (Padding & Batching)
Esta función toma una lista de archivos, los nivela con silencio y los procesa en un solo Tensor masivo.

In [ ]:
def process_batch(file_paths):
    waveforms = []
    metadata = []
    max_len = 0
    
    # 1. Carga y nivelación de longitudes (Padding)
    for path in file_paths:
        wav, sr = torchaudio.load(path)
        if sr != SAMPLE_RATE:
            wav = torchaudio.transforms.Resample(sr, SAMPLE_RATE)(wav)
        
        wav = torch.mean(wav, dim=0) # Mono
        waveforms.append(wav)
        max_len = max(max_len, wav.shape[0])
        metadata.append({"name": os.path.basename(path), "len": wav.shape[0]})
    
    # Crear el Tensor de Lote (Padding con ceros)
    batch_wav = torch.zeros(len(waveforms), max_len).to(device)
    for i, wav in enumerate(waveforms):
        batch_wav[i, :wav.shape[0]] = wav
        
    # 2. PROCESAMIENTO PARALELO EN GPU
    t0 = time.time()
    with torch.no_grad():
        # Calculamos TODO el lote en una sola llamada
        mel_outputs = mel_gpu(batch_wav)
        cqt_outputs = cqt_gpu(batch_wav)
        
        # Convertir a dB
        mel_db = torchaudio.transforms.AmplitudeToDB()(mel_outputs).cpu().numpy()
        cqt_db = torchaudio.transforms.AmplitudeToDB()(torch.abs(cqt_outputs)).cpu().numpy()
    
    batch_time = time.time() - t0
    
    # 3. Separación y Guardado de resultados individuales
    batch_results = []
    for i in range(len(file_paths)):
        fname = metadata[i]['name']
        # Recortar el padding para el análisis final
        orig_len_frames = int(metadata[i]['len'] / 512) # Aproximación de frames
        
        # Visualización técnica
        plt.figure(figsize=(15, 8))
        plt.subplot(2, 1, 1)
        librosa.display.specshow(mel_db[i], x_axis='time', y_axis='mel', sr=SAMPLE_RATE, cmap='magma')
        plt.title(f"BATCH SPECTROGRAM: {fname} | Lote: {len(file_paths)} pistas | Tiempo Lote: {batch_time:.3f}s")
        
        plt.subplot(2, 1, 2)
        librosa.display.specshow(cqt_db[i], x_axis='time', y_axis='cqt_note', sr=SAMPLE_RATE, cmap='inferno')
        
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, 'plots', f"{fname}_batch.png"))
        plt.close()
        
        batch_results.append({"file": fname, "batch_speed": batch_time / len(file_paths)})
        
    return batch_results

## 🚀 3. Ejecución por Lotes

In [ ]:
DATASET_PATH = '/kaggle/input/datasets/danieldobles/slavic-songs'
if not os.path.exists(DATASET_PATH): DATASET_PATH = 'Slavic Data_Set'
all_files = [os.path.join(DATASET_PATH, f) for f in os.listdir(DATASET_PATH) if f.endswith(('.mp3', '.wav')) and f != 'REF.flac' ]

results = []
print(f"🔥 Iniciando Trituradora de Matrices (Batch Size: {BATCH_SIZE})")

for i in tqdm(range(0, len(all_files), BATCH_SIZE), desc="Procesando Lotes"):
    batch_paths = all_files[i : i + BATCH_SIZE]
    try:
        res = process_batch(batch_paths)
        results.append(res)
    except Exception as e:
        print(f"❌ Error en lote: {e}")

print("\n🏆 ¡Procesamiento Masivo Completado!")
shutil.make_archive('MIR_BATCH_GPU_RESULTS', 'zip', OUTPUT_DIR)
display(FileLink('MIR_BATCH_GPU_RESULTS.zip', result_html_prefix="🚀 DESCARGA EL RESULTADO MASIVO: "))